## Lesson 2:  Building Agents to Simplify Vector Datbase Query Tasks

Now that we understand vector databases and how to make queries, lets build some helpers!

## Install Pydantic

We will use Pydantic to build agents to help us with our vector searches. But first, we will build some simple agents with straighforward tasks

In [ ]:
#!pip install pydantic-ai

## Create an agent that likes to keep the conversation going

In [ ]:
from pydantic_ai import Agent

agent = Agent(
    model="gpt-4o-mini",
    #TODO- Give your new agent some fun instructions
)

In [ ]:
result = await agent.run("What is Data Science Dojo?")
print(result.output)

## Create an agent with memory

In [ ]:
result = await agent.run("I am most intersted in agnets")
print(result.output)

Let' help our agent out and give him some memory.

In [ ]:
result = await agent.run("What is Data Science Dojo?")
print(result.output)

In [ ]:
result = await agent.run("Yes, I would like to pursue data science education!", message_history=result.all_messages())
print(result.output)

## Reconnecting to Wevaite

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


print("Weaviate URL:", WEAVIATE_URL)
print("Weaviate API Key:", WEAVIATE_KEY[:10])
print("OpenAI API Key:", OPENAI_API_KEY[:10])

In [ ]:
import weaviate
from weaviate.classes.init import Auth

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),
    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY
    },
)

print("Client ready:", client.is_ready())

In [ ]:
# Check our collection from before
contracts = client.collections.use("FinancialContract")
contracts_config = contracts.config.get()

print(articles_config)

In [ ]:
from pydantic_ai import Agent

# Simple query optimizer and search agent
#Todo- make another agent like before

async def search_(user_query):
    # Optimize and search in one step
    result = await search_agent.run(f"Optimize this query for contract search: {user_query}")
    optimized_query = result.output

    # Search the database
    #Todo- now run your agent and give him a task
    # Show results
    for i, contract in enumerate(response.objects):
        print(f"Contract {i+1}: {contract.properties['contract_text']}")

    return response.objects



In [ ]:
# Test it
results = await search_("I need job contract info")

In [ ]:
from weaviate.classes.query import Filter
from datetime import datetime, timezone

# Get collection properties
collection_config = contracts.config.get()
properties = {prop.name: str(prop.data_type) for prop in collection_config.properties}

# Smart filtering agent that creates filters automatically
smart_filter_agent = Agent(
    model="openai:gpt-4o-mini",
    instructions=f"""
    You analyze queries and automatically create Weaviate filters.
    Collection properties: {properties}
    
    Based on the user query, generate Python code that creates Filter objects.
    Only return the filter code, nothing else.
    
    Examples:
    - For "Jane Doe contracts": Filter.by_property("author").equal("Jane Doe")
    - For "recent employment contracts": Filter.by_property("contract_type").equal("employment contract") & 
Filter.by_property("date").greater_than(datetime(2023, 6, 1, tzinfo=timezone.utc))
    - For "short contracts": Filter.by_property("contract_length").less_than(2)
    
    If no filters needed, return: None
    """,
)

async def auto_filtered_search(user_query):
    # Get filter code from agent
    filter_result = await smart_filter_agent.run(f"Create filters for: {user_query}")
    filter_code = filter_result.output.strip()

    print(f"Query: {user_query}")
    print(f"Generated filter: {filter_code}")

    # Execute the filter code
    query_filters = None
    if filter_code != "None":
        try:
            query_filters = eval(filter_code)
        except:
            print("Filter creation failed, searching without filters")

    # Search with auto-generated filters
    response = contracts.query.near_text(
        query=user_query,
        filters=query_filters,
        limit=3
    )

    print(f"\nFound {len(response.objects)} contracts:")
    for i, contract in enumerate(response.objects):
        print(f"Contract {i+1}: {contract.properties['contract_type']} by {contract.properties['author']} on {contract.properties['date']}")

    return response.objects

In [ ]:
results = await auto_filtered_search("Show me Jane Doe's recent employment contracts after 2022")

In [ ]:
client.close()